[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ML-Village/Giza-Orion-Experiments/blob/main/notebooks/pytorch-mnist.ipynb)

In [1]:
#@title Install libs
!pip install torch torchvision

In [2]:
!python --version

Python 3.10.12


In [8]:
#@title Import Dependencies
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [11]:
torch.__version__

'2.1.0+cu121'

In [13]:
#@title Set ze Params

args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=1000
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset.
args['lr']=0.01 #Learning rate is how fast it will decend.
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=False

#vs
# input_size = 784 # img_size = (28,28) ---> 28*28=784 in total
# hidden_size = 500 # number of nodes at hidden layer
# num_classes = 10 # number of output classes discrete range [0,9]
# num_epochs = 20 # number of times which the entire dataset is passed throughout the model
# batch_size = 100 # the size of input data took for one iteration
# lr = 1e-3 # size of step

In [15]:
#@title Load ze data

train_loader = torch.utils.data.DataLoader(

    # downloads training set
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)) #normalizing
                   ])),

    batch_size=args['batch_size'], shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(

    # downloads test set
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)) #normalizing
                   ])),


    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

In [16]:
#@title Define ze ModeL

class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities.
        return F.log_softmax(x, dim=1)

# class Net(nn.Module):
#   def __init__(self, input_size, hidden_size, num_classes):
#     super(Net,self).__init__()
#     self.fc1 = nn.Linear(input_size, hidden_size)
#     self.relu = nn.ReLU()
#     self.fc2 = nn.Linear(hidden_size, num_classes)

#   def forward(self,x):
#     out = self.fc1(x)
#     out = self.relu(out)
#     out = self.fc2(out)
#     return out

In [24]:
#@title Define training routine/flow

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable.
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch.
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically.
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
#@title Train ze Model
model = Net()

if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])
# loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam( net.parameters(), lr=lr)

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318971
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.305999
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.290581
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.290898
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.280602
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.278978


<ipython-input-24-c021442c44eb>:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 2.2560, Accuracy: 2156/10000 (22%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.266870
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.249291
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.246460
Train Epoch: 2 [30000/60000 (50%)]	Loss: 2.224019
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.194281
Train Epoch: 2 [50000/60000 (83%)]	Loss: 2.163766

Test set: Average loss: 2.0671, Accuracy: 5482/10000 (55%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.130990
Train Epoch: 3 [10000/60000 (17%)]	Loss: 2.068974
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.990030
Train Epoch: 3 [30000/60000 (50%)]	Loss: 1.875356
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.770804
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.652321

Test set: Average loss: 1.2439, Accuracy: 7438/10000 (74%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.602181
Train Epoch: 4 [10000/60000 (17%)]	Loss: 1.469216
Train Epoch: 4 [20000/60000 (33%)]	Loss: 1.360229
Train Epoch: 4 [30000/60000 (50%)]	Loss: 1.275462
Train Epoch: 4 [40000/60000 (67%

In [ ]:
#@title Evaluating the accuracy of the model

correct = 0
total = 0
for batch_idx, (images, labels) in enumerate(test_loader):
    if args['cuda']:
        images, labels = images.cuda(), labels.cuda()
    #Variables in Pytorch are differenciable.
    images, labels = Variable(images), Variable(labels)

    output = model(images)
    #_, predicted = torch.max(output,1)
    predicted =output.data.max(1, keepdim=True)[1]
    # correct += (predicted == labels).sum()
    correct += predicted.eq(labels.data.view_as(predicted)).long().cpu().sum()

    total += labels.size(0)

print('Accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

In [29]:
torch.save(model.state_dict(), 'checkpoint.pth')

In [30]:
#@title Save MoDEL

# upload external file before import
from google.colab import files

# download checkpoint file
files.download('checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
#@title Export to ze Onnx MoDEL
!pip install onnx onnxscript

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 550.7/550.7 kB 5.8 MB/s eta 0:00:00


In [38]:
inputs, classes = next(iter(test_loader))

In [39]:
onnx_program = torch.onnx.dynamo_export(model, inputs)

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/exporter.py:130: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [41]:
onnx_program.save("mnisttorch.onnx")
# download onnx file
files.download('mnisttorch.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>